# Initial Imports & Setup

In [2]:
#!/usr/bin/env python3

# Data Science Modules
import numpy as np
import pandas as pd
import altair as alt
#from selenium import webdriver

# Python Modules
import glob as glob
import os
import shutil
import datetime as datetime
from multiprocessing import Pool

#%load_ext blackcellmagic

# Lookup Data

* **./files/names.csv**: CSV File with Provider/Clinic Names, etc. Names is used to determine active providers/clinics. (So if there's no name, it wont get evaluated later -- say, for example, after Everson Closes.)
* **./files/metrics.csv**: CSV File with Metrics information (Display Name, Category, Target, etc.)

In [3]:
# Read Lookup files into dataframes for mapping
names = pd.read_csv("./files/names.csv", index_col="MeridiosName")
metrics = pd.read_csv(
    "./files/metrics.csv", index_col="MeridiosMetric", dtype={"MeridiosMetric": object}
)

# Import Data from /data folder.

This makes a dataframe in a 'Tidy Data' format.

* **./data/(Zero-Padded Date at beginning of CSVs from Meridos).csv**: Files from Automatic weekly report from Meridios

In [4]:
# The base dataframe, df, get's transformed into different data subsets.
df = pd.DataFrame()

# Read in all the data files and append to dataframe
files = glob.glob("./data/*.csv")
for file in files:
    file_df = pd.read_csv(file, usecols=["NAME", "Metricname", "SeenNum", "SeenDenom"])

    # Lookup column names
    file_df["Name"] = file_df.NAME.map(names.Name)
    file_df["Type"] = file_df.NAME.map(names.Type)
    file_df["Clinic"] = file_df.NAME.map(names.Clinic)
    file_df["Metric"] = file_df.Metricname.map(metrics.Metric)

    # Show anything not in names or metrics
    #if len(df) is not 0:
    #    print(file_df[~file_df.Name.isin(names["Name"])].Name)
    #    #print(file_df[~file_df.Metric.isin(metrics["Metric"])].Metric)
    
    # Create FCN Summary for each metric
    clinics_df = file_df[(file_df["Type"] == "Clinic")]
    fcn_data = []
    for metric in clinics_df.Metric.unique():
        metric_df = clinics_df[(clinics_df["Metric"] == metric)]
        fcn_numerator = metric_df.SeenNum.sum()
        fcn_denominator = metric_df.SeenDenom.sum()
        dataline = ("", "", fcn_numerator, fcn_denominator, "FCN", "FCN", "FCN", metric)
        fcn_data.append(dataline)
    fcn_df = pd.DataFrame.from_records(fcn_data, columns=metric_df.columns)
    file_df = file_df.append(fcn_df, ignore_index=True)

    # Who are we kidding with precision? Round to Basis points
    # Not actually percentage (but rather than * 100 leaving for formatting
    # in visualization.)
    file_df["Percentage"] = round(file_df["SeenNum"] / file_df["SeenDenom"], 4)
        
    # Zero Padded Dates with Dots. Like: "03.15.2018" We're using the filename
    # as the source of the date column because Meridios exports haven't always
    # had the date-time data correct.
    filename_parts = file[7:-4].split(" ")
    if len(str(filename_parts[0])) is 10:
        file_df["Date"] = datetime.datetime.strptime(filename_parts[0], "%m.%d.%Y")

        # Now delete the 'raw' columns from import so dataframe is smaller.
        file_df.drop(
            ["NAME", "Metricname", "SeenNum", "SeenDenom"],
            axis=1,
            inplace=True,
        )
        df = df.append(file_df)
    else:
        # We're only appending data that had Zero Padded Date at
        # beginning of filename.
        raise ValueError("CSV Filename should have Zero Padded Date.")

# Dataframe Review

The dataframe is the dataset for all the various graphs. We can inspect it various ways, if needed. (Used for development, not really validation.)

In [4]:
# df.dtypes
# df.info()
# df.describe()
# df.columns
# df.Metric.unique()
# df.head()
# metrics.head()

## Validations:

* Do new CSV files include new MeridiosName, New Metric items? (Has the underlying report changed? Such as when edited provider name.)
* Should we keep a list of already imported data and do some message to clarify after new data is available?
* Percentages under 0% and over 100% are report errors

Currently no validations. But probably should automate checks that new data from report is accurate. 

# Provider Graph Function

In [5]:
def make_individual_metric_chart(metric, name, savefolder):
    """
    Makes a chart for a single metric and a single provider.
    
    Assumes: dataframe 'df' that has all the data from CSVs
    Assumes: dataframes 'names' and 'metrics' for lookups
    """

    provider_df = df[
        (df["Metric"] == metric) & (df["Type"] == "Individual") & (df["Name"] == name)
    ]

    provider_df = provider_df.drop(["Name", "Type", "Clinic", "Metric"], axis=1)

    # Lookup clinic from the provider name in the names dataframe.
    # Make a comparison dataframe.
    clinic_name = names[names.Name == name].iloc[0].Clinic
    clinic_df = df[(df["Metric"] == metric) & (df["Name"] == clinic_name)]
    clinic_df = clinic_df.drop(["Name", "Type", "Clinic", "Metric"], axis=1)

    fcn_df = df[(df["Metric"] == metric) & (df["Type"] == "FCN")]
    fcn_df = fcn_df.drop(["Name", "Type", "Clinic", "Metric"], axis=1)

    # Lookup the metric target -- not all metrics have a target.
    metric_target = metrics[metrics.Metric == metric].iloc[0].Target

    # If there's a target value, we'll make a rule on graph.
    if metric_target:
        metricdf = pd.DataFrame([{"TargetValue": metric_target, "Title": "Target"}])

    # Make a Current dataframe to use for Strip Chart.
    current_metric = df[
        (df["Metric"] == metric)
        & (df["Type"] == "Individual")
        & (df["Date"] == current_date)
    ]
    current_metric = current_metric.drop(["Type", "Clinic", "Metric", "Date"], axis=1)

    # Altair Graphs to combine.
    provider_progress_line = (
        alt.Chart(provider_df)
        .mark_line(strokeWidth=4)
        .encode(
            alt.X("Date:T", title=""),
            alt.Y(
                "Percentage:Q",
                axis=alt.Axis(format="%", title=""),
                scale=alt.Scale(domain=(0, 1)),
            ),
            # color=alt.Color("Name:N", legend=None, ),
            color=alt.ColorValue("#9467bd"),
        )
        .properties(width=200, height=200)
    )

    clinic_progress_line = (
        alt.Chart(clinic_df)
        .mark_line(strokeWidth=2)
        .encode(
            alt.X("Date:T", title=""),
            alt.Y("Percentage:Q"),
            color=alt.ColorValue("#ffbb78"),
        )
    )

    fcn_progress_line = (
        alt.Chart(fcn_df)
        .mark_line(strokeWidth=2)
        .encode(
            alt.X("Date:T", title=""),
            alt.Y("Percentage:Q"),
            color=alt.ColorValue("#aec7e8"),
        )
    )

    if metric_target:
        metric_target_rule = (
            alt.Chart(metricdf)
            .mark_rule(strokeWidth=1, strokeDash=[4, 2])
            .encode(
                y="TargetValue:Q",
                color=alt.ColorValue("#2ca02c"),
            )
        )

    fcn_current_strip_chart = (
        alt.Chart(current_metric)
        .mark_tick(color="#ddd")
        .encode(
            alt.Y(
                "Percentage:Q",
                axis=alt.Axis(format="%", title="", labels=False),
                scale=alt.Scale(domain=(0, 1)),
            )
        )
        .properties(height=200)
    )

    provider_highlight_strip = (
        alt.Chart(current_metric)
        .mark_tick()
        .encode(
            alt.Y("Percentage:Q"),
            opacity=alt.value("1.0"),
            color=alt.ColorValue("#9467bd"),
        )
        .transform_filter(alt.FieldEqualPredicate(field="Name", equal=name))
    )

    provider_percent = (
        provider_highlight_strip.mark_text(
            align="left", baseline="middle", dx=15, size=20
        )
        .encode(text=alt.Text("Percentage:Q", format=".2%"))
        .transform_filter(alt.FieldEqualPredicate(field="Name", equal=name))
    )

    if metric_target:
        chart = (
            metric_target_rule
            + fcn_progress_line
            + clinic_progress_line
            + provider_progress_line
            | (fcn_current_strip_chart + provider_highlight_strip + provider_percent)
        )
        chart.save(savefolder + str(metric).replace(" ", "_") + ".json", scale_factor=2)
    else:
        chart = fcn_progress_line + clinic_progress_line + provider_progress_line | (
            fcn_current_strip_chart + provider_highlight_strip + provider_percent
        )
        chart.save(savefolder + str(metric).replace(" ", "_") + ".json", scale_factor=2)


# Clinic Graph Function

In [6]:
def make_clinic_metric_chart(metric, clinic_name, savefolder):
    """
    Makes a chart for a single metric and a clinic.
    
    Assumes: dataframe 'df' that has all the data from CSVs
    Assumes: dataframes 'names' and 'metrics' for lookups
    """

    clinic_df = df[(df["Metric"] == metric) & (df["Name"] == clinic_name)]
    clinic_df = clinic_df.drop(["Name", "Type", "Clinic", "Metric"], axis=1)

    fcn_df = df[(df["Metric"] == metric) & (df["Type"] == "FCN")]
    fcn_df = fcn_df.drop(["Name", "Type", "Clinic", "Metric"], axis=1)

    metric_target = metrics[metrics.Metric == metric].iloc[0].Target

    # If there's a target value, we'll make a rule on graph.
    if metric_target:
        metricdf = pd.DataFrame([{"TargetValue": metric_target, "Title": "Target"}])

    current_metric = df[
        (df["Metric"] == metric)
        & (df["Type"] == "Clinic")
        & (df["Date"] == current_date)
    ]

    # Altair Graphs to combine.
    clinic_progress_line = (
        alt.Chart(clinic_df)
        .mark_line(strokeWidth=4)
        .encode(
            alt.X("Date:T", title=""),
            alt.Y(
                "Percentage:Q",
                axis=alt.Axis(format="%", title=""),
                scale=alt.Scale(domain=(0, 1)),
            ),
            color=alt.ColorValue("#ff7f0e"),
        )
        .properties(width=200, height=200)
    )

    fcn_progress_line = (
        alt.Chart(fcn_df)
        .mark_line(strokeWidth=2)
        .encode(
            alt.X("Date:T", title=""),
            alt.Y(
                "Percentage:Q",
                axis=alt.Axis(format="%", title=""),
                scale=alt.Scale(domain=(0, 1)),
            ),
            color=alt.ColorValue("#aec7e8"),
        )
    )

    if metric_target:
        metric_target_rule = (
            alt.Chart(metricdf)
            .mark_rule(strokeWidth=1, strokeDash=[4, 2])
            .encode(
                y="TargetValue:Q",
                color=alt.ColorValue("#2ca02c"),
            )
        )

    clinic_providers = sorted(
        single_providers[single_providers.Clinic == clinic_name].Name.unique(),
        key=lambda x: x.split(" ")[1],
    )

    current_metric = df[
        (df["Metric"] == metric)
        & (df["Date"] == current_date)
        & (df["Name"].isin(clinic_providers))
    ]
    current_metric = current_metric.drop(["Type", "Clinic", "Metric", "Date"], axis=1)

    start_date = min(clinic_df["Date"])
    start_metric = df[
        (df["Metric"] == metric)
        & (df["Date"] == start_date)
        & (df["Name"].isin(clinic_providers))
    ]
    start_metric = start_metric.drop(["Type", "Clinic", "Metric", "Date"], axis=1)
    start_and_current = pd.concat([start_metric, current_metric])

    ranged_dot = (
        alt.Chart(start_and_current)
        .mark_line(color="#c5b")
        .encode(
            alt.Y(
                "Percentage:Q",
                axis=alt.Axis(format="%", title=""),
                scale=alt.Scale(domain=(0, 1)),
            ),
            alt.X("Name:N", axis=alt.Axis(title=""), sort=clinic_providers),
            detail="Name:N",
        )
        .properties(height=200)
    )

    ranged_dot += (
        alt.Chart(current_metric)
        .mark_point(size=100, opacity=1, filled=True, color="#9467bd")
        .encode(alt.Y("Percentage:Q"), alt.X("Name:N", sort=clinic_providers))
    )

    ranged_dot_rule = (
        alt.Chart(metricdf)
        .mark_rule(strokeWidth=1, strokeDash=[4, 2])
        .encode(y="TargetValue:Q", color=alt.value("#2ca02c"))
    )

    if metric_target:
        chart = (
            metric_target_rule + fcn_progress_line + clinic_progress_line 
        ) | ranged_dot + ranged_dot_rule
        chart.save(savefolder + str(metric).replace(" ", "_") + ".json", scale_factor=2)
    else:
        chart = (fcn_progress_line + clinic_progress_line) | ranged_dot
        chart.save(savefolder + str(metric).replace(" ", "_") + ".json", scale_factor=2)


# FCN Graph Function

In [7]:
def make_fcn_metric_chart(metric, savefolder):
    """
    Makes a chart for a single metric for FCN.
    
    Assumes: dataframe 'df' that has all the data from CSVs
    Assumes: dataframes 'names' and 'metrics' for lookups
    """

    fcn_df = df[(df["Metric"] == metric) & (df["Type"] == "FCN")]
    fcn_df = fcn_df.drop(["Name", "Type", "Clinic", "Metric"], axis=1)

    # If there's a target value, we'll make a rule on graph.
    metric_target = metrics[metrics.Metric == metric].iloc[0].Target
    if metric_target:
        metricdf = pd.DataFrame([{"TargetValue": metric_target, "Title": "Target"}])

    # Altair Graphs to combine.
    fcn_progress_line = (
        alt.Chart(fcn_df)
        .mark_line(strokeWidth=4)
        .encode(
            alt.X("Date:T", title=""),
            alt.Y(
                "Percentage:Q",
                axis=alt.Axis(format="%", title=""),
                scale=alt.Scale(domain=(0, 1)),
            ),
            color=alt.ColorValue("#1F77B4"),
        )
        .properties(width=200, height=200)
    )

    if metric_target:
        metric_target_rule = (
            alt.Chart(metricdf)
            .mark_rule(strokeWidth=1, strokeDash=[4, 2])
            .encode(
                y="TargetValue:Q",
                color=alt.ColorValue("#2ca02c"),
            )
        )

    current_metric = df[
        (df["Metric"] == metric)
        & (df["Date"] == current_date)
        & (df["Type"] == "Clinic")
    ]
    current_metric = current_metric.drop(["Type", "Clinic", "Metric", "Date"], axis=1)

    start_date = min(fcn_df["Date"])
    start_metric = df[
        (df["Metric"] == metric) & (df["Date"] == start_date) & (df["Type"] == "Clinic")
    ]
    start_metric = start_metric.drop(["Type", "Clinic", "Metric", "Date"], axis=1)

    start_and_current = pd.concat([start_metric, current_metric])

    ranged_dot = (
        alt.Chart(start_and_current)
        .mark_line(color="#ffbb78")
        .encode(
            alt.Y(
                "Percentage:Q",
                axis=alt.Axis(format="%", title=""),
                scale=alt.Scale(domain=(0, 1)),
            ),
            alt.X("Name:N", axis=alt.Axis(title="")),
            detail="Name:N",
            tooltip=["Name", "Percentage"],
        )
        .properties(height=200)
    )

    ranged_dot += (
        alt.Chart(current_metric)
        .mark_point(size=100, opacity=1, filled=True, color="#FF7F0E")
        .encode(alt.Y("Percentage:Q"), alt.X("Name:N"), tooltip=["Name", "Percentage"])
    )

    ranged_dot_rule = (
        alt.Chart(metricdf)
        .mark_rule(strokeWidth=1, strokeDash=[4, 2])
        .encode(y="TargetValue:Q", color=alt.value("#2ca02c"))
    )

    if metric_target:
        chart = (metric_target_rule + fcn_progress_line) | ranged_dot + ranged_dot_rule
        chart.save(savefolder + str(metric).replace(" ", "_") + ".json", scale_factor=2)
    else:
        chart = (fcn_progress_line) | ranged_dot
        chart.save(savefolder + str(metric).replace(" ", "_") + ".json", scale_factor=2)


# Prepping for the website generation

In [8]:
# Need to just do active individuals, main metrics
single_providers = names[(names["Type"] == "Individual")]

# Sorted by Last Name 
sorted_single_provider_names = sorted(
    single_providers.Name.unique(), key=lambda x: x.split(" ")[1]
)
clinics = sorted(set(df[(df["Type"] == "Clinic")].Name.unique()))
main_metrics = sorted(set(metrics[(metrics["Main"] == "Main")].Metric.unique()))
current_date = max(df["Date"])
current_date_string = current_date.strftime("%m/%d/%Y")

def savefolder(name):
    foldername = str(name).replace(" ", "_")
    if not os.path.exists("./docs/" + foldername):
        os.makedirs("./docs/" + foldername)
    return "./docs/" + foldername + "/"


def create_individual_metrics(name):
    for metric in main_metrics:
        make_individual_metric_chart(metric, name, savefolder(name))


def create_clinic_metrics(clinic_name):
    for metric in main_metrics:
        chart = make_clinic_metric_chart(metric, clinic_name, savefolder(clinic_name))

# Import Image Files

In [9]:
FCN_logo = "./files/pictures/logo.png"
if os.path.isfile(FCN_logo):
    if not os.path.exists("./docs/pictures/"):
        os.makedirs("./docs/pictures/")
    shutil.copyfile(FCN_logo, "./docs/pictures/logo.png")

favicon = "./files/pictures/favicon.ico"
if os.path.isfile(favicon):
    shutil.copyfile(favicon, "./docs/favicon.ico")

for name in sorted_single_provider_names:
    provider_picture = "./files/pictures/" + str(name).replace(" ", "_") + ".JPG"
    if os.path.isfile(provider_picture):
        shutil.copyfile(
            provider_picture, "./docs/pictures/" + str(name).replace(" ", "_") + ".JPG"
        )
    else:
        print("File not found:", provider_picture)


File not found: ./files/pictures/Patricia_Payne-Zajac.JPG
File not found: ./files/pictures/April_Wakefield-Pagels.JPG
File not found: ./files/pictures/Sloane_Winkes.JPG


# Navbar Function

In [10]:
def make_navbar(provider):

    clinic_name = names[names.Name == provider].iloc[0].Clinic
    # Type of Providers: Individual, Clinic, FCN
    type = names[names.Name == provider].iloc[0].Type

    same_clinic_providers = sorted(
        single_providers[single_providers.Clinic == clinic_name].Name.unique(),
        key=lambda x: x.split(" ")[1],
    )
    navbar = (
        '<div uk-sticky class="uk-inline uk-background-default uk-text-lead uk-text-middle" style="padding-top:1em;">\n'
    )

    # FCN Logo back to root unless FCN page.
    if type != "FCN":
        navbar += (
            '<a href="../index.html"><img src="../pictures/logo.png" width="39" height="64"></a>\n'
        )
    else:
        navbar += '<img src="../pictures/logo.png" width="39" height="64">\n'

    navbar += '<div class="uk-inline">Patients seeing:&nbsp;</div>\n'

    # If inidivudual provider, use Picture
    if type == "Individual":
        navbar += (
            '<img src="'
            + "../pictures/"
            + str(provider).replace(" ", "_")
            + ".JPG"
            + '" width="64" height="64" class="uk-border-circle">&nbsp;\n'
        )

    # Now The Single Provider Links
    if type == "Individual":
        navbar += (
            f'<div class="uk-inline uk-text-bold" style="color:#9467bd">{provider}'
        )

    else:
        navbar += '<div class="uk-inline" style="color:#9467bd">Providers'

    if type == "Individual" or "Clinic":
        navbar += (
            '&nbsp;&#9660;</div>\n<div uk-dropdown><ul class="uk-nav uk-dropdown-nav">\n'
        )
        for clinic_provider in same_clinic_providers:
            if provider == clinic_provider:
                navbar += f'<li class="uk-active">{provider}</li>\n'
            else:
                navbar += (
                    '<li><a href="../'
                    + str(clinic_provider).replace(" ", "_")
                    + '/index.html">'
                    + clinic_provider
                    + "</a></li>\n"
                )
    else:
        navbar += "</div>\n"
    if type == "Individual" or "Clinic":
        navbar += "</ul></div>\n"

    navbar += "&nbsp;@&nbsp;\n"

    if type == "Clinic":
        navbar += (
            f'<div class="uk-inline uk-text-bold" style="color:#ff7f0e">{clinic_name}'
        )
    else:
        navbar += f'<div class="uk-inline" style="color:#ff7f0e">{clinic_name}'

    navbar += '&#9660;</div>\n<div uk-dropdown><ul class="uk-nav uk-dropdown-nav">\n'

    for clinic in clinics:
        if clinic == clinic_name:
            navbar += (
                f'<li class="uk-active"><a href="../{str(clinic).replace(" ", "_")}/index.html">{clinic}</a></li>\n'
            )
        else:
            navbar += (
                f'<li><a href="../{str(clinic).replace(" ", "_")}/index.html">{clinic}</a></li>\n'
            )
    navbar += "</ul></div>\n&nbsp;@&nbsp;\n"

    if type != "FCN":
        navbar += (
            '<a class="uk-inline" style="color:#1F77B4" href="../FCN/index.html">FCN</a>\n'
        )
    else:
        navbar += (
            '<a class="uk-inline uk-text-bold" style="color:#1F77B4" href="../FCN/">FCN</a>\n'
        )

    navbar += "</div>\n"
    return navbar


# Generate HTMLs for Providers, Clinics, and FCN

In [11]:
for provider in sorted_single_provider_names:
    navbar = make_navbar(provider)
    with open("./files/index.html", "r") as file:
        filedata = file.read()
    filedata = filedata.replace("{{Navbar}}", navbar)
    filedata = filedata.replace("{{{Current Date}}}", current_date_string)
    with open(savefolder(provider) + "index.html", "w+") as file:
        file.write(filedata)

for provider in clinics:
    navbar = make_navbar(provider)
    with open("./files/index-clinic.html", "r") as file:
        filedata = file.read()
    filedata = filedata.replace("{{Navbar}}", navbar)
    filedata = filedata.replace("{{{Current Date}}}", current_date_string)
    with open(savefolder(provider) + "index.html", "w+") as file:
        file.write(filedata)

provider = "FCN"
navbar = make_navbar(provider)
with open("./files/index-clinic.html", "r") as file:
    filedata = file.read()
filedata = filedata.replace("{{Navbar}}", navbar)
filedata = filedata.replace("{{{Current Date}}}", current_date_string)
with open(savefolder(provider) + "index.html", "w+") as file:
    file.write(filedata)

# Generate Site Root HTML

In [12]:
# Base HTML File
root_index_clinic = (
    '<div uk-filter="target: .js-filter"><ul class="uk-subnav uk-subnav-pill">\n'
)

for clinic in clinics:
    root_index_clinic += (
        '<li uk-filter-control=".tag-'
        + clinic
        + '"><a href="#">'
        + clinic
        + "</a></li>\n"
    )
root_index_clinic += "</ul>"

provider_index_cards = (
    '<ul class="js-filter uk-grid-match uk-card-small uk-text-center" uk-grid>\n'
)

for name in sorted_single_provider_names:
    provider_icon = (
        '<img class="uk-align-center uk-border-circle" src="'
        + "./pictures/"
        + str(name).replace(" ", "_")
        + ".JPG"
        + '" width="64" height="64" class="">'
    )
    provider_index_cards += (
        '<li class="tag-'
        + names[names.Name == name].iloc[0].Clinic
        + '"><a class="uk-align-center" href="./'
        + str(name).replace(" ", "_")
        + '/"><div class="uk-card uk-width-medium uk-card-hover uk-card-default uk-card-body">'
        + provider_icon
        + name
        + "</div></a></li>\n"
    )
provider_index_cards += "</ul>"

with open("./files/index-base.html", "r") as file:
    filedata = file.read()
filedata = filedata.replace("{{{Clinics}}}", root_index_clinic)
filedata = filedata.replace("{{{Provider-Index-Cards}}}", provider_index_cards)
filedata = filedata.replace("{{{Current Date}}}", current_date_string)
with open("docs/" + "index.html", "w+") as file:
    file.write(filedata)

# Make Charts with Multiprocessing

In [13]:
pool = Pool()
pool.map(create_individual_metrics, sorted_single_provider_names)
pool.close()
pool.join()

In [14]:
pool2 = Pool()
pool2.map(create_clinic_metrics, clinics)
pool2.close()
pool2.join()

In [15]:
for metric in main_metrics:
        chart = make_fcn_metric_chart(metric, savefolder("FCN"))